# Imports

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()

from utils.funcs import compute_metrics
from utils.plots import plot_two_coin_distributions



Loading BokehJS ...

# Calculations

In [2]:
n = 50
p0 = 0.5
p1 = 0.6
alpha = 0.05
k_obs = 32

metrics = compute_metrics(n=n, p0=p0, p1=p1, alpha=alpha, k_obs=k_obs)
metrics


{'alpha': 0.05,
 'beta': 0.7629431215000944,
 'power': 0.23705687849990564,
 'p_value': 0.06490864707227217}

### Visualizations (coin perspective)

We toss an *unknown* coin **n** times and observe **k** heads.

- **H0 (fair coin)**: the coin is fair, so \(p = 0.5\). Under H0, \(k \sim \text{Binomial}(n, 0.5)\).
- **H1 (biased coin)**: the coin is biased, so \(p = p_1\) (for example 0.6). Under H1, \(k \sim \text{Binomial}(n, p_1)\).

On the plot we show *two discrete distributions* (PMFs):
- **Blue (H0)**: how many heads we would expect if the coin is fair.
- **Orange (H1)**: how many heads we would expect if the coin is biased.

#### How we make a decision
We choose a significance level **alpha** and define a **rejection region** under H0.
- The **red points** are outcomes where we would **reject H0** ("the coin is not fair").
- If the coin is actually fair (H0 is true), rejecting H0 is a **Type I error**. Its probability is about **alpha**.

If the coin is actually biased (H1 is true), we can still fail to detect it:
- The **green points** are outcomes where we would **keep H0** even though H1 is true.
- This probability is **beta** (Type II error).
- **Power = 1 - beta**.

#### What p-value means on the plot
For the observed **k** (vertical black line) we compute a **two-sided p-value** under H0.
- The **purple points** are outcomes "at least as extreme" as the observed one under H0 (the p-value tails).
- Decision rule by p-value: **reject H0 if p-value <= threshold**.

Usually the threshold equals **alpha**, but you can change them independently to see how the decision changes.

In [3]:
rng = np.random.default_rng(42)

p0 = 0.5  # fair coin (H0)
p1 = 0.6  # biased coin (H1 example)

alpha = 0.05
p_value_threshold = 0.05

plots_by_n = []
for n in [20, 50, 100]:
    # Imagine the *true* coin is biased with p=p1 and we observe k heads.
    k_obs = int(rng.binomial(n=n, p=p1))

    plots_by_n.append(
        plot_two_coin_distributions(
            n=n,
            p0=p0,
            p1=p1,
            alpha=alpha,
            k_obs=k_obs,
            p_value_threshold=p_value_threshold,
            title=f"Observed coin: n={n}, k={k_obs}; alpha={alpha}",
        )
    )

show(column(*plots_by_n))

# Same n and observation, different alpha (changes the rejection region and beta).
alpha_compare = row(
    plot_two_coin_distributions(
        n=50,
        p0=p0,
        p1=p1,
        alpha=0.10,
        k_obs=30,
        p_value_threshold=0.10,
        title="n=50, alpha=0.10",
    ),
    plot_two_coin_distributions(
        n=50,
        p0=p0,
        p1=p1,
        alpha=0.01,
        k_obs=30,
        p_value_threshold=0.01,
        title="n=50, alpha=0.01",
    ),
)

show(alpha_compare)
